# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,25.82,69,0,2.06,RE,1710992827
1,1,dunaivtsi,48.8891,26.8564,0.75,97,100,5.42,UA,1710992827
2,2,albany,42.6001,-73.9662,-2.02,68,39,1.34,US,1710992783
3,3,grytviken,-54.2811,-36.5092,2.20,87,100,3.30,GS,1710992828
4,4,margaret river,-33.9500,115.0667,24.46,38,0,3.86,AU,1710992599


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
#https://hvplot.holoviz.org/user_guide/Geographic_Data.html
#https://stackoverflow.com/questions/70994916/hvplot-points-with-geo-true-does-not-show-the-map
# %%capture --no-display

# Configure the map plot


mappy=city_data_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=.65,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                hover_cols=['Country','Cloudiness','Wind Speed'])

# Display the map
mappy


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
#criteria:temperature between 21 to 30,Humidity is less than 80%, windspeed is less then 4m/s, cloudiness<75

crit_df=city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<30) & 
                         (city_data_df['Humidity']<80) & (city_data_df['Wind Speed']<4)
                        &(city_data_df['Cloudiness']<75)].dropna()


# Display sample data
crit_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,25.82,69,0,2.06,RE,1710992827
4,4,margaret river,-33.9500,115.0667,24.46,38,0,3.86,AU,1710992599
9,9,meulaboh,4.1363,96.1285,29.89,67,48,1.24,ID,1710992831
12,12,taoudenni,22.6783,-3.9836,23.72,8,3,2.37,ML,1710992832
66,66,kone,-21.0595,164.8658,28.77,72,65,3.39,NC,1710992852
82,82,alo,7.4178,6.7371,27.79,77,47,1.43,NG,1710992857
100,100,ta`u,-14.2336,-169.5144,29.05,73,31,2.11,AS,1710992863
107,107,corcoran,36.0980,-119.5604,21.16,43,36,3.98,US,1710992865
111,111,avarua,-21.2078,-159.7750,29.03,79,20,2.06,CK,1710992867
134,134,taunggyi,20.7833,97.0333,28.30,27,1,1.35,MM,1710992876


#### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=crit_df.loc[:,['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''



# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,69,
4,margaret river,AU,-33.9500,115.0667,38,
9,meulaboh,ID,4.1363,96.1285,67,
12,taoudenni,ML,22.6783,-3.9836,8,
66,kone,NC,-21.0595,164.8658,72,
82,alo,NG,7.4178,6.7371,77,
100,ta`u,AS,-14.2336,-169.5144,73,
107,corcoran,US,36.0980,-119.5604,43,
111,avarua,CK,-21.2078,-159.7750,79,
134,taunggyi,MM,20.7833,97.0333,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'
limit=1

params = {'categories':categories,
          'apiKey':geoapify_key,'limit':limit}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
margaret river - nearest hotel: Margaret River Hotel
meulaboh - nearest hotel: Hotel Meuligou
taoudenni - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
alo - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
corcoran - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
taunggyi - nearest hotel: Golden Island Cottages
yung shue wan - nearest hotel: 浪濤軒酒店 Concerto Inn
minab - nearest hotel: هتل صدف
sinabang - nearest hotel: No hotel found
bhainsdehi - nearest hotel: No hotel found
kajuru - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
bela - nearest hotel: Hotel City Palace
gummi - nearest hotel: No hotel found
besuki - nearest hotel: Utama raya pantai situbondo
san jose del cabo - nearest hotel: Hotel Posada Terranova
luwuk - nearest hotel: No hotel found
maracaju - nearest hotel: Hotel Cachoeira
pompeia - nearest hotel: Hotel Pousada Va

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
4,margaret river,AU,-33.9500,115.0667,38,Margaret River Hotel
9,meulaboh,ID,4.1363,96.1285,67,Hotel Meuligou
12,taoudenni,ML,22.6783,-3.9836,8,No hotel found
66,kone,NC,-21.0595,164.8658,72,Pacifik Appartels
82,alo,NG,7.4178,6.7371,77,No hotel found
100,ta`u,AS,-14.2336,-169.5144,73,No hotel found
107,corcoran,US,36.0980,-119.5604,43,No hotel found
111,avarua,CK,-21.2078,-159.7750,79,Paradise Inn
134,taunggyi,MM,20.7833,97.0333,27,Golden Island Cottages


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display
#https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot#:~:text=You%20can%20use%20keyword%20hover_cols%20to%20add%20additional%20columns%20to%20your%20hover.&text=Additional%20columns%20to%20add%20to,indexes%20if%20use_index%20is%20True).

# Configure the map plot
hotel_mappy=hotel_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=1,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                   xlim=(-180,180),ylim=(-60,60),hover_cols=['Country','Hotel Name'])


# Display the map
hotel_mappy

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)